<a href="https://colab.research.google.com/github/JonJonesBR/MESTRE_RPG_GEMINI/blob/main/RPG_AUDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q google-generativeai edge-tts playsound nest_asyncio gradio Pillow

import gradio as gr
import google.generativeai as genai
import time
import os
import io
import sys
import asyncio
import re

# Configuração para permitir loops asyncio aninhados
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("nest_asyncio aplicado.")
except ImportError:
    print("AVISO: nest_asyncio não encontrado.")
except RuntimeError as e:
    if "cannot apply nest_asyncio" in str(e) or "loop is already running" in str(e):
        print(f"nest_asyncio: {e}. Provavelmente já está gerenciado pelo ambiente (ex: Gradio).")
    else:
        print(f"AVISO: nest_asyncio: {e}")

_edge_tts_available = False
try:
    import edge_tts
    _edge_tts_available = True
except ImportError:
    print("AVISO: Biblioteca edge-tts não encontrada. A funcionalidade de voz não estará disponível.")

API_KEY_FILENAME = ".gemini_api_key.txt"

DURATION_OPTIONS_CONFIG = {
    "1": {"name": "Curta (objetivo direto)", "id": "curta", "turns": 10},
    "2": {"name": "Média (mais exploração)", "id": "media", "turns": 20},
    "3": {"name": "Longa (desenvolvimento e clímax)", "id": "longa", "turns": 40}
}

RPG_THEMES_CONFIG = {
    "medieval": "Medieval Clássico",
    "fantasia": "Alta Fantasia",
    "futurista": "Futurista (Sci-Fi)",
    "cyberpunk": "Cyberpunk Noir",
    "steampunk": "Steampunk",
    "velho_oeste": "Velho Oeste",
    "pos_apoc": "Pós-Apocalíptico"
}

THEME_CLASSES = {
    "medieval": ["Cavaleiro(a) Honrado(a)", "Mago(a) Erudito(a)", "Arqueiro(a) Furtivo(a)", "Clérigo(a) Devoto(a)", "Bardo(a) Viajante"],
    "fantasia": ["Guerreiro(a) Élfico(a)", "Feiticeira Elemental", "Ladino(a) Halfling", "Paladino(a) Draconato(a)", "Druida Metamorfo"],
    "futurista": ["Piloto(a) de Mecha", "Engenheiro(a) Cibernético(a)", "Agente Secreto(a) Biônico(a)", "Explorador(a) Estelar", "Diplomata Galáctico(a)"],
    "cyberpunk": ["Samurai das Ruas", "Netrunner (Hacker)", "Detetive Bio-modificado(a)", "Traficante de Informações", "Médico(a) Clandestino(a)"],
    "steampunk": ["Inventor(a) Genial", "Caçador(a) de Autômatos", "Aeronauta Destemido(a)", "Detetive a Vapor", "Engenhateiro(a) Real"],
    "velho_oeste": ["Pistoleiro(a) Solitário(a)", "Caçador(a) de Recompensas", "Médico(a) Itinerante", "Jogador(a) de Cartas", "Xerife da Fronteira"],
    "pos_apoc": ["Sobrevivente Nato(a)", "Batedor(a) de Ermos", "Mecânico(a) de Sucata", "Líder de Facção", "Mutante Errante"]
}


model_text = None
MODEL_NAME_TEXT = "gemini-1.5-flash-latest"

generation_config_text = {"temperature": 0.85, "top_p": 0.95, "top_k": 40}
safety_settings_text = [
    {"category": c, "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
    for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]
]

async def _speak_text_async_gradio(text, voice_name="pt-BR-ThalitaNeural", rate="+0%", pitch="+0Hz"):
    if not _edge_tts_available or not text or not text.strip(): return None
    text_cleaned = re.sub(r'[\*#_`]', ' ', text)
    text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    if not text_cleaned: return None
    try:
        communicate = edge_tts.Communicate(text_cleaned, voice_name, rate=rate, pitch=pitch)
        audio_bytes_io = io.BytesIO()
        async for chunk in communicate.stream():
            if chunk["type"] == "audio": audio_bytes_io.write(chunk["data"])
        audio_bytes_io.seek(0)

        if audio_bytes_io.getbuffer().nbytes > 0:
            temp_filename = f"temp_audio_{int(time.time()*1000)}.mp3"
            with open(temp_filename, "wb") as f: f.write(audio_bytes_io.getvalue())
            return temp_filename
        return None
    except Exception as e_tts:
        print(f"[TTS Edge Erro Geral: {e_tts}]")
        return None

async def speak_text_for_gradio(text, voice_name="pt-BR-ThalitaNeural"):
    if not _edge_tts_available or not text or not text.strip(): return None
    return await _speak_text_async_gradio(text, voice_name)

def configure_gemini_api_gradio_direct(api_key_input):
    global model_text
    feedback_list = []
    api_key_val = api_key_input.strip() if api_key_input else ""

    if not api_key_val or not api_key_val.startswith("AIza") or len(api_key_val) <= 20:
        feedback_list.append("Formato de chave API inválido.")
        if os.path.exists(API_KEY_FILENAME):
            try: os.remove(API_KEY_FILENAME); feedback_list.append(f"Arquivo '{API_KEY_FILENAME}' removido.")
            except Exception as e_rem: feedback_list.append(f"Erro ao remover '{API_KEY_FILENAME}': {e_rem}")
        return "\n".join(feedback_list), gr.update(visible=True), gr.update(visible=False)

    try:
        with open(API_KEY_FILENAME, "w") as f: f.write(api_key_val)
        feedback_list.append(f"Chave API salva em '{API_KEY_FILENAME}'.")
    except Exception as e:
        feedback_list.append(f"Não foi possível salvar a chave API: {e}.")

    try:
        genai.configure(api_key=api_key_val)
        model_text = genai.GenerativeModel(
            model_name=MODEL_NAME_TEXT,
            generation_config=generation_config_text,
            safety_settings=safety_settings_text
        )
        feedback_list.append(f"API Gemini configurada. Modelo de Texto '{model_text.model_name}' pronto.")
        return "\n".join(feedback_list), gr.update(visible=False), gr.update(visible=True)
    except Exception as e:
        feedback_list.append(f"Erro CRÍTICO ao configurar API Gemini (Modelo: {MODEL_NAME_TEXT}): {e}")
        if os.path.exists(API_KEY_FILENAME):
            try: os.remove(API_KEY_FILENAME); feedback_list.append(f"Arquivo '{API_KEY_FILENAME}' removido devido a erro crítico.")
            except Exception as e_rem: feedback_list.append(f"Erro ao remover chave: {e_rem}")
        model_text = None
        return "\n".join(feedback_list), gr.update(visible=True), gr.update(visible=False)


def load_api_key_on_startup_gradio():
    global model_text
    api_key_loaded_val = ""
    feedback = []
    api_section_vis = gr.update(visible=True)
    game_setup_vis = gr.update(visible=False)

    if os.path.exists(API_KEY_FILENAME):
        try:
            with open(API_KEY_FILENAME, "r") as f: api_key_loaded_val = f.read().strip()
            if api_key_loaded_val and api_key_loaded_val.startswith("AIza") and len(api_key_loaded_val) > 20:
                feedback.append("Chave API carregada de arquivo local.")
                config_feedback_str, api_section_vis, game_setup_vis = configure_gemini_api_gradio_direct(api_key_loaded_val)
                feedback.append(config_feedback_str)
            else:
                if os.path.exists(API_KEY_FILENAME): os.remove(API_KEY_FILENAME)
                feedback.append(f"Chave em '{API_KEY_FILENAME}' inválida, arquivo removido.")
                api_key_loaded_val = ""
        except Exception as e:
            feedback.append(f"Erro ao ler '{API_KEY_FILENAME}': {e}")
            api_key_loaded_val = ""

    if not model_text:
        if not any("API Gemini configurada" in s for s in feedback) and not any("Erro CRÍTICO" in s for s in feedback):
            feedback.append("Insira sua chave API Gemini.")
        api_section_vis = gr.update(visible=True)
        game_setup_vis = gr.update(visible=False)

    initial_classes = THEME_CLASSES.get("medieval", [])
    initial_class_radio_update = gr.update(choices=initial_classes, value=initial_classes[0] if initial_classes else None)

    return "\n".join(feedback), api_section_vis, game_setup_vis, api_key_loaded_val, initial_class_radio_update


def mestre_responde(prompt_para_o_mestre):
    global model_text
    if not model_text: return "Erro: Modelo de Texto Gemini não inicializado. Verifique a API Key."
    try:
        # print(f"--- PROMPT PARA O MESTRE ---\n{prompt_para_o_mestre}\n---------------------------")
        response = model_text.generate_content(prompt_para_o_mestre)
        # print(f"--- RESPOSTA DO MESTRE ---\n{response.text}\n---------------------------")
        text_resp = response.text
        return text_resp
    except Exception as e:
        return f"Erro na API Gemini (Texto): {e}"

def parse_initial_setup_from_gemini(response_text):
    data = {
        "current_objective": "Explorar o desconhecido e descobrir seu propósito.",
        "current_location": "Um local misterioso e evocativo.",
        "player_inventory": ["Um item básico de sobrevivência", "Uma lembrança peculiar"],
        "character_lore": "Um indivíduo com um passado nebuloso, buscando um futuro mais claro.",
        "world_lore": "Um mundo vasto e cheio de segredos, perigos e maravilhas."
    }
    try:
        if not response_text or not response_text.strip():
            print("Resposta do setup inicial vazia, usando defaults.")
            return data

        lines = response_text.strip().split('\n')
        for line in lines:
            if ":" not in line: continue
            prefix, value = line.split(":", 1)
            value = value.strip()
            prefix_upper = prefix.upper().strip()

            if prefix_upper == "LORE_MUNDO": data["world_lore"] = value
            elif prefix_upper == "LORE_PERSONAGEM": data["character_lore"] = value
            elif prefix_upper in ["LOCALIZAÇÃO", "LOCALIZACAO"]: data["current_location"] = value
            elif prefix_upper == "OBJETIVO": data["current_objective"] = value
            elif prefix_upper in ["INVENTÁRIO", "INVENTARIO"]:
                items_str = value.replace('[','').replace(']','')
                parsed_items = [item.strip().strip("'\"") for item in items_str.split(',') if item.strip()]
                if parsed_items: data["player_inventory"] = parsed_items

    except Exception as e:
        print(f"Erro no parse_initial_setup: {e}. Resposta recebida:\n---\n{response_text}\n---")
    return data

async def iniciar_aventura_gradio(player_name_input, player_class_choice, custom_class_input, rpg_theme_key, duration_choice_key, gs_dict, ch_list):
    gs = {}
    ch = []
    initial_chatbot_history = []
    log_setup_feedback = []

    gs["player_name"] = player_name_input.strip() or "Aventureiro(a) Anônimo(a)"
    log_setup_feedback.append(f"Nome do Jogador: {gs['player_name']}")

    gs["player_class"] = custom_class_input.strip() or player_class_choice
    if not gs["player_class"]:
        default_classes_for_theme = THEME_CLASSES.get(rpg_theme_key, THEME_CLASSES["medieval"])
        gs["player_class"] = default_classes_for_theme[0] if default_classes_for_theme else "Aventureiro(a) Genérico(a)"
    log_setup_feedback.append(f"Classe: {gs['player_class']}")


    gs["rpg_theme"] = RPG_THEMES_CONFIG.get(rpg_theme_key, "Medieval Clássico")
    log_setup_feedback.append(f"Tema da Aventura: {gs['rpg_theme']}")

    opt = DURATION_OPTIONS_CONFIG.get(duration_choice_key, DURATION_OPTIONS_CONFIG["1"])
    gs.update({"chosen_duration_id": opt['id'], "chosen_duration_name": opt['name'], "max_turns": opt['turns'], "current_turn": 0})
    log_setup_feedback.append(f"Duração Estimada: {gs['chosen_duration_name']}")

    prompt_setup = f"""Você é um mestre de RPG criando a base para uma nova aventura.
Tema Escolhido: {gs['rpg_theme']}
Nome do Jogador: {gs['player_name']}
Classe do Jogador: {gs['player_class']}
Duração da Aventura (tipo): {gs['chosen_duration_name']}

Gere os seguintes elementos para o início da aventura. Seja criativo, conciso e adequado ao tema e à classe do jogador.
Responda APENAS com os seguintes prefixos, cada um em uma nova linha, seguido pelo conteúdo:
LORE_MUNDO: [Uma breve e envolvente descrição do mundo e sua atmosfera principal, alinhada com o tema.]
LORE_PERSONAGEM: [Uma breve história de origem ou motivação para o personagem do jogador ({gs['player_class']}), conectada ao mundo e ao tema.]
LOCALIZAÇÃO: [O nome e uma breve descrição do local inicial onde a aventura começa. Este é o local para a PARTE 2 da narração inicial.]
OBJETIVO: [O primeiro objetivo claro e conciso para o jogador.]
INVENTÁRIO: [Liste de 2 a 4 itens iniciais que o personagem ({gs['player_class']}) possui, temáticos. Ex: item1, item2, item3]

Exemplo de formato de resposta (NÃO use este conteúdo, apenas o formato e os prefixos):
LORE_MUNDO: A metrópole cyberpunk de Neo-Veridia, onde a chuva ácida nunca cessa e os arranha-céus perfuram as nuvens de poluição.
LORE_PERSONAGEM: Um Netrunner ({gs['player_class']}) fugindo de uma mega-corporação após um hack que deu errado, buscando refúgio nos subterrâneos da cidade.
LOCALIZAÇÃO: Beco dos Esquecidos, um distrito mal-iluminado nos confins da cidade baixa.
OBJETIVO: Encontrar o contrabandista de informações conhecido como "Espectro".
INVENTÁRIO: Cyberdeck antigo, jaqueta com capuz, alguns créditos quebrados, um datachip misterioso.
"""
    resposta_setup = mestre_responde(prompt_setup)
    resposta_setup_content = str(resposta_setup) if resposta_setup is not None else "O mestre não conseguiu gerar o setup."


    if "Erro: Modelo de Texto Gemini não inicializado" in resposta_setup_content or "Erro na API Gemini" in resposta_setup_content :
        log_setup_feedback.append(f"Falha crítica ao gerar setup: {resposta_setup_content}")
        initial_chatbot_history.append({"role": "assistant", "content": resposta_setup_content})
        return gs, ch, initial_chatbot_history, "\n".join(log_setup_feedback), None, gr.update(visible=True), gr.update(visible=False)

    parsed_data = parse_initial_setup_from_gemini(resposta_setup_content)
    gs.update(parsed_data)

    log_setup_feedback.append(f"--- DETALHES DA AVENTURA GERADOS PELA IA (Referência) ---")
    log_setup_feedback.append(f"Lore do Mundo: {gs.get('world_lore', 'Não definido.')}")
    log_setup_feedback.append(f"Sua História (Lore): {gs.get('character_lore', 'Não definido.')}")
    log_setup_feedback.append(f"Objetivo Inicial: {gs['current_objective']}")
    log_setup_feedback.append(f"Localização Inicial (para a cena): {gs['current_location']}")
    log_setup_feedback.append(f"Inventário Inicial: {', '.join(gs['player_inventory'])}")

    # MODIFICAÇÃO AQUI: Novo prompt_contexto
    prompt_contexto = f"""Você é o Mestre de um RPG Textual, preparando a introdução para o jogador {gs['player_name']}.

Informações Base Geradas para esta Aventura (use-as para construir sua narração):
- Tema: {gs['rpg_theme']}
- Jogador: {gs['player_name']} (Classe: {gs['player_class']})
- Lore do Mundo: {gs.get('world_lore', 'Um mundo vasto e misterioso.')}
- Lore do Personagem ({gs['player_name']}): {gs.get('character_lore', 'Um aventureiro com um passado intrigante.')}
- Objetivo Inicial do Personagem: {gs['current_objective']}
- Localização Inicial da Cena da Aventura: {gs['current_location']}
- Inventário Inicial do Personagem: {', '.join(gs['player_inventory'])}

Instruções para sua resposta (combine tudo em uma única narração fluida e envolvente):

PARTE 1: APRESENTAÇÃO DO MUNDO E PERSONAGEM
1.  Comece com uma saudação ou uma introdução que situe o jogador {gs['player_name']}.
2.  Descreva brevemente, em tom narrativo e atmosférico, o mundo em que a aventura se passa, baseando-se na "Lore do Mundo" fornecida. (1-2 parágrafos)
3.  Apresente o personagem {gs['player_name']}, descrevendo um pouco de quem ele(a) é, sua situação atual ou uma característica marcante, utilizando a "Lore do Personagem". (1-2 parágrafos)
4.  Mencione de forma integrada à apresentação do personagem o "Objetivo Inicial" que o move.
5.  Descreva brevemente, como parte da apresentação do personagem (talvez enquanto ele se prepara ou reflete), os itens que ele(a) carrega (o "Inventário Inicial"). Não precisa ser uma lista formal, mas sim integrado. Ex: "Você verifica seu [item1], uma herança de família, e ajusta a [item2] em suas costas, pronto para o que vier..."

PARTE 2: INÍCIO DA CENA DA AVENTURA
1.  Após a apresentação do personagem e seu contexto, faça uma transição suave para a cena de início da aventura.
2.  Descreva a "Localização Inicial da Cena da Aventura" de forma imersiva e detalhada, usando os sentidos (o que se vê, ouve, cheira?). (2-3 parágrafos curtos)
3.  Coloque o personagem {gs['player_name']} diretamente na ação ou em uma situação que exija uma decisão, observação ou interação imediata.
4.  Apresente um pequeno desafio, mistério, encontro ou gancho que se conecte ao "Objetivo Inicial".

PARTE 3: CHAMADA PARA AÇÃO
1.  Termine sua narração completa convidando o jogador {gs['player_name']} à sua primeira ação (ex: "O que você faz?", "Qual seu próximo passo?", "Como você reage?").

IMPORTANTE:
-   Sua resposta deve ser uma única peça de texto contínua e bem escrita.
-   Use um tom de mestre de RPG envolvente, descritivo e criativo.
-   NÃO repita os prefixos (LORE_MUNDO:, OBJETIVO:, etc.) na sua resposta. Apenas use as informações fornecidas para construir a narrativa.
-   A Parte 1 (Apresentação) deve fluir naturalmente para a Parte 2 (Cena).
"""
    desc_inicial = mestre_responde(prompt_contexto)
    desc_inicial_content = str(desc_inicial) if desc_inicial is not None else "O mestre não conseguiu iniciar a narração com a apresentação da lore."
    audio_path = None

    if "Erro:" in desc_inicial_content or not desc_inicial_content.strip() or desc_inicial_content == "O mestre não conseguiu iniciar a narração com a apresentação da lore.":
        final_desc = f"O Mestre (IA) falhou ao criar a introdução completa da aventura: {desc_inicial_content or 'Resposta vazia.'}"
        initial_chatbot_history.append({"role": "assistant", "content": final_desc})
        audio_path = await speak_text_for_gradio(final_desc)
        gs["player_name"] = None
        return gs, ch, initial_chatbot_history, "\n".join(log_setup_feedback), audio_path, gr.update(visible=True), gr.update(visible=False)

    initial_chatbot_history.append({"role": "assistant", "content": desc_inicial_content})
    audio_path = await speak_text_for_gradio(desc_inicial_content)
    gs["current_situation"] = desc_inicial_content # A situação atual é a narração completa

    # O histórico para o modelo Gemini agora reflete a narração completa que ele mesmo gerou.
    ch.append({'role': 'system',
               'parts': [f"Contexto Geral da Aventura (Tema: {gs['rpg_theme']}): Mundo: {gs.get('world_lore')}. Personagem Base: {gs.get('character_lore')} (Classe: {gs['player_class']}). Objetivo Inicial Geral: {gs['current_objective']}."]})
    ch.append({'role': 'model', 'parts': [desc_inicial_content]}) # A primeira fala do mestre é essa introdução completa.

    return gs, ch, initial_chatbot_history, "\n".join(log_setup_feedback), audio_path, gr.update(visible=False), gr.update(visible=True)

async def rodada_do_jogo_gradio(acao_jogador_input, gs_dict, ch_list, current_chatbot_hist_msgs, game_ended_flag_val):
    gs = gs_dict.copy()
    ch = list(ch_list)
    updated_chatbot_hist = list(current_chatbot_hist_msgs) if current_chatbot_hist_msgs is not None else []


    if not acao_jogador_input.strip():
        yield gs, ch, updated_chatbot_hist, None, gr.update(interactive=True), "", game_ended_flag_val
        return

    updated_chatbot_hist.append({"role": "user", "content": str(acao_jogador_input)})
    updated_chatbot_hist.append({"role": "assistant", "content": "Mestre (Gemini) está ponderando sobre sua ação..."})
    yield gs, ch, updated_chatbot_hist, None, gr.update(interactive=False), "", game_ended_flag_val

    gs["current_turn"] = gs.get("current_turn", 0) + 1
    audio_mestre_path = None
    jogo_terminou_nesta_rodada = game_ended_flag_val

    if acao_jogador_input.lower() == 'sair':
        msg_sair = f"A jornada de {gs.get('player_name', 'O Aventureiro')} chega ao fim por decisão própria. Obrigado por jogar!"
        updated_chatbot_hist[-1] = {"role": "assistant", "content": msg_sair}
        audio_mestre_path = await speak_text_for_gradio(msg_sair)
        jogo_terminou_nesta_rodada = True
        yield gs, ch, updated_chatbot_hist, audio_mestre_path, gr.update(interactive=False), "", jogo_terminou_nesta_rodada
        return

    gs["previous_player_actions"] = (gs.get("previous_player_actions", []) + [str(acao_jogador_input)])[-3:]

    instrucao_duracao = ""
    ct, mt, cdn = gs.get('current_turn',0), gs.get('max_turns',1), gs.get('chosen_duration_name','')
    if mt > 0 :
        if ct == mt: instrucao_duracao = f"ATENÇÃO: Este é o ÚLTIMO TURNO ({ct}/{mt}) da aventura '{cdn}'! Crie uma conclusão impactante para o arco narrativo atual."
        elif ct > mt * 0.75: instrucao_duracao = f"AVISO: A aventura '{cdn}' está se aproximando do fim (Turno {ct}/{mt}). Guie a narrativa para um clímax ou resolução."

    hist_fmt_parts = []
    # Usar ch (histórico para o modelo Gemini) para construir o prompt, não o chatbot_hist (que pode ter mensagens de "analisando")
    for entry in ch[-6:]: # Pegar as últimas 6 interações do histórico do modelo
        role_label = "Jogador" if entry['role'] == 'user' else ("Contexto do Sistema" if entry['role'] == 'system' else "Mestre")
        # 'parts' é uma lista, pegar o primeiro elemento.
        content_parts = entry.get('parts', [])
        content = content_parts[0] if content_parts and isinstance(content_parts[0], str) else str(content_parts)

        if entry['role'] != 'system' : # Não precisa mostrar o prompt do sistema para si mesmo.
             hist_fmt_parts.append(f"{role_label}: {content}")
    hist_fmt = "\n".join(hist_fmt_parts)

    prompt_mestre = f"""Você é o Mestre de um RPG Textual.
Tema da Aventura: {gs.get('rpg_theme', 'Não especificado')}.
Jogador: {gs.get('player_name', 'Aventureiro')} (Classe: {gs.get('player_class', 'Desconhecida')}).
Duração da Aventura: {cdn} (total de {mt} turnos), Turno Atual: {ct}. {instrucao_duracao}

SOBRE O MUNDO (Lore): {gs.get('world_lore', 'O mundo é um mistério a ser descoberto.')}
SOBRE O PERSONAGEM (Lore): {gs.get('character_lore', 'Um aventureiro com um passado enigmático e um futuro incerto.')}

HISTÓRICO RECENTE DA CONVERSA (Mestre e Jogador):
{hist_fmt or 'Esta é a primeira ação do jogador após a introdução.'}

SITUAÇÃO ATUAL NA AVENTURA (descrita na sua última narração completa):
{gs.get('current_situation', 'A aventura está prestes a começar ou o estado anterior não foi registrado.')}

AÇÃO DO JOGADOR AGORA: '{str(acao_jogador_input)}'

INVENTÁRIO ATUAL DO JOGADOR: {', '.join(gs.get('player_inventory', ['Nada']))}
OBJETIVO ATUAL DO JOGADOR: {gs.get('current_objective', 'Descobrir o que fazer.')}

SUAS INSTRUÇÕES COMO MESTRE (LEIA COM ATENÇÃO):
1.  Narre a consequência da ação do jogador de forma criativa, detalhada e envolvente, mantendo a consistência com o tema, a lore, a classe ({gs.get('player_class')}) e a situação atual.
2.  Descreva o ambiente, NPCs (se houver), diálogos e acontecimentos de forma imersiva.
3.  Se a ação do jogador resultar em uma MUDANÇA CLARA E SIGNIFICATIVA no objetivo principal, APENAS nessa situação, adicione uma linha EXATAMENTE assim AO FINAL da sua narração principal:
    NOVO_OBJETIVO: [Descreva o novo objetivo aqui]
4.  Se a ação do jogador resultar em uma MUDANÇA CLARA E SIGNIFICATIVA para uma nova área ou localização principal, APENAS nessa situação, adicione uma linha EXATAMENTE assim AO FINAL da sua narração principal:
    NOVA_LOCALIZAÇÃO: [Descreva a nova localização aqui]
5.  Se a ação do jogador resultar em uma MUDANÇA CLARA E SIGNIFICATIVA no inventário (itens importantes ganhos ou perdidos), APENAS nessa situação, liste o NOVO inventário COMPLETO do jogador em uma linha EXATAMENTE assim AO FINAL da sua narração principal:
    NOVO_INVENTÁRIO: [item1, item2, item3]
6.  Sua narração principal deve ser o foco (2 a 4 parágrafos). As linhas de atualização (NOVO_OBJETIVO, etc.) são apenas para registrar mudanças de estado importantes e devem vir por último, se aplicável. NÃO as use para pequenas alterações.
7.  NÃO repita o inventário, objetivo ou localização na sua narração principal A MENOS que seja orgânico para a história. As linhas de atualização no final servem para o sistema.
8.  Termine sua resposta com uma pergunta ou uma deixa que convide o jogador à próxima ação (a menos que seja o final da aventura, conforme {instrucao_duracao}).
9.  Seja criativo e faça o jogador se sentir parte da história! Mantenha o suspense e a progressão.
"""
    resp_mestre_raw = mestre_responde(prompt_mestre)
    resp_mestre_content = str(resp_mestre_raw) if resp_mestre_raw is not None else "O mestre não respondeu."


    if "Erro: Modelo de Texto Gemini não inicializado" in resp_mestre_content or "Erro na API Gemini" in resp_mestre_content :
        updated_chatbot_hist[-1] = {"role": "assistant", "content": resp_mestre_content}
        audio_mestre_path = await speak_text_for_gradio(resp_mestre_content)
        yield gs, ch, updated_chatbot_hist, audio_mestre_path, gr.update(interactive=True), "", True
        return

    narrative_to_display_and_speak = resp_mestre_content # Default, será ajustado abaixo
    if "Erro:" in resp_mestre_content or resp_mestre_content == "O mestre não respondeu.":
        narrative_to_display_and_speak = f"Mestre parece confuso e com problemas técnicos: {resp_mestre_content}"
        audio_mestre_path = await speak_text_for_gradio(narrative_to_display_and_speak)
        gs["current_turn"] -= 1
    else:
        lines = resp_mestre_content.split('\n')
        final_narrative_lines = []
        parsed_updates_for_log = []

        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("NOVO_OBJETIVO:"):
                new_obj = stripped_line.split(":", 1)[1].strip()
                if new_obj:
                    gs["current_objective"] = new_obj
                    parsed_updates_for_log.append(f"Objetivo atualizado para: {new_obj}")
            elif stripped_line.startswith("NOVA_LOCALIZAÇÃO:"):
                new_loc = stripped_line.split(":", 1)[1].strip()
                if new_loc:
                    gs["current_location"] = new_loc
                    parsed_updates_for_log.append(f"Localização atualizada para: {new_loc}")
            elif stripped_line.startswith("NOVO_INVENTÁRIO:"):
                items_str = stripped_line.split(":", 1)[1].strip()
                parsed_items = [item.strip() for item in items_str.split(',') if item.strip()]
                if parsed_items:
                    gs["player_inventory"] = parsed_items
                    parsed_updates_for_log.append(f"Inventário atualizado para: {', '.join(parsed_items)}")
                elif not items_str :
                    gs["player_inventory"] = []
                    parsed_updates_for_log.append(f"Inventário agora está vazio.")
            else:
                final_narrative_lines.append(line)

        narrative_only_for_tts_and_situation = "\n".join(final_narrative_lines).strip()

        # Construir a mensagem para o chatbot
        if not narrative_only_for_tts_and_situation and parsed_updates_for_log:
            narrative_to_display_and_speak = "O Mestre registra as mudanças no ambiente e no seu personagem."
            if parsed_updates_for_log:
                 narrative_to_display_and_speak += "\n\n" + "\n".join([f"<i>({update})</i>" for update in parsed_updates_for_log])
        elif not narrative_only_for_tts_and_situation:
             narrative_to_display_and_speak = "O Mestre contempla em silêncio, talvez aguardando um momento mais oportuno para falar..."
        else:
            narrative_to_display_and_speak = narrative_only_for_tts_and_situation
            if parsed_updates_for_log:
                 narrative_to_display_and_speak += "\n\n" + "\n".join([f"<i>({update})</i>" for update in parsed_updates_for_log])

        audio_mestre_path = await speak_text_for_gradio(narrative_only_for_tts_and_situation)
        gs["current_situation"] = narrative_only_for_tts_and_situation

        ch.append({'role': 'user', 'parts': [str(acao_jogador_input)]})
        ch.append({'role': 'model', 'parts': [narrative_to_display_and_speak]}) # Histórico do modelo com a narrativa formatada (incluindo updates em itálico)

    updated_chatbot_hist[-1] = {"role": "assistant", "content": narrative_to_display_and_speak}

    if gs.get("current_turn", 0) >= gs.get("max_turns", 1) and not jogo_terminou_nesta_rodada:
        final_msg_turns = f"\nA aventura '{cdn}' chegou ao seu limite de turnos ({gs['current_turn']}/{gs['max_turns']})."
        if not any(keyword in narrative_to_display_and_speak.lower() for keyword in ["fim", "conclusão", "termina", "acabou"]):
            updated_chatbot_hist.append({"role": "assistant", "content": final_msg_turns})
        jogo_terminou_nesta_rodada = True

    yield gs, ch, updated_chatbot_hist, audio_mestre_path, gr.update(interactive=not jogo_terminou_nesta_rodada), "", jogo_terminou_nesta_rodada

# --- Interface Gradio ---
with gr.Blocks(theme=gr.themes.Soft(), title="RPG com Gemini Aprimorado") as demo:
    game_state_store = gr.State({})
    conversation_history_store = gr.State([])
    game_ended_flag = gr.State(False)

    gr.Markdown("# 🎲 RPG Textual Aprimorado com IA Gemini e Voz 🎙️")
    gr.Markdown("Crie seu personagem, escolha um tema e embarque em aventuras narradas por uma Inteligência Artificial!")

    with gr.Tabs():
        with gr.TabItem("⚙️ Configuração e Jogo 🎮"):
            with gr.Row():
                with gr.Column(scale=1):
                    with gr.Column(visible=True) as api_key_section:
                        gr.Markdown("## 1. Chave API Gemini")
                        gr.Markdown("Para jogar, você precisa de uma Chave API do Google AI Studio (Gemini). [Obtenha sua chave aqui!](https://aistudio.google.com/app/apikey)")
                        api_key_input_gradio = gr.Textbox(label="Sua Chave API do Gemini", placeholder="Cole sua chave AIza...", type="password", lines=1)
                        submit_api_key_button = gr.Button("🔑 Configurar API e Modelo")
                        api_key_feedback = gr.Textbox(label="Feedback da Configuração API", lines=3, interactive=False, max_lines=5)

                    with gr.Column(visible=False) as game_setup_block:
                        gr.Markdown("## 2. Crie Sua Aventura")
                        player_name_gradio = gr.Textbox(label="Nome do Aventureiro(a)", placeholder="Ex: Elara, Kael, Jax")

                        rpg_theme_options_display = [(name, key) for key, name in RPG_THEMES_CONFIG.items()]
                        rpg_theme_choice_gradio = gr.Dropdown(rpg_theme_options_display, label="Escolha o Tema do RPG", value="medieval")

                        player_class_choice_gradio = gr.Radio(label="Escolha sua Classe Principal")

                        custom_class_gradio = gr.Textbox(label="Ou personalize sua classe (opcional)", placeholder="Ex: Caçador de Relíquias, Tecnomante")

                        duration_options_display = [(v['name'], k) for k, v in DURATION_OPTIONS_CONFIG.items()]
                        duration_choice_gradio = gr.Radio(duration_options_display, label="Duração da Aventura", value="1")

                        start_adventure_button = gr.Button("🚀 Iniciar Aventura!")
                        game_setup_feedback = gr.Textbox(label="Log de Criação da Aventura e Lore (Referência)", lines=8, interactive=False, max_lines=15)
                        audio_output_setup = gr.Audio(label="🎧 Narração Inicial do Mestre", autoplay=True)

                with gr.Column(scale=2, visible=False) as main_game_block:
                    gr.Markdown("## 3. Sua Aventura se Desenrola...")
                    chatbot_component = gr.Chatbot(
                        label="Narrativa da Aventura", height=600, type='messages',
                        bubble_full_width=False,
                        avatar_images=(None, "https://www.gstatic.com/lamda/images/gemini_sparkle_v002_darkμορφή.svg"))

                    player_action_input = gr.Textbox(label="O que você faz a seguir?", placeholder="Digite sua ação ou 'sair' para terminar...", show_label=True, lines=2)
                    submit_action_button = gr.Button("➡️ Enviar Ação")
                    audio_output_game = gr.Audio(label="🎧 Narração do Mestre", autoplay=True)

    # --- Lógica dos Eventos Gradio ---
    def update_class_choices_on_theme_change(theme_key):
        classes = THEME_CLASSES.get(theme_key, [])
        if not classes:
            classes = THEME_CLASSES.get("medieval", ["Classe Padrão"])
        return gr.update(choices=classes, value=classes[0] if classes else None)

    rpg_theme_choice_gradio.change(
        fn=update_class_choices_on_theme_change,
        inputs=[rpg_theme_choice_gradio],
        outputs=[player_class_choice_gradio]
    )

    demo.load(
        load_api_key_on_startup_gradio,
        inputs=[],
        outputs=[api_key_feedback, api_key_section, game_setup_block, api_key_input_gradio, player_class_choice_gradio])

    submit_api_key_button.click(
        configure_gemini_api_gradio_direct,
        inputs=[api_key_input_gradio],
        outputs=[api_key_feedback, api_key_section, game_setup_block])

    def reset_game_state_for_new_adventure():
        return {}, [], False, None, "", None, None

    start_adventure_button.click(
        reset_game_state_for_new_adventure,
        outputs=[game_state_store, conversation_history_store, game_ended_flag, chatbot_component, player_action_input, audio_output_setup, audio_output_game]
    ).then(
        iniciar_aventura_gradio,
        inputs=[player_name_gradio, player_class_choice_gradio, custom_class_gradio, rpg_theme_choice_gradio, duration_choice_gradio, game_state_store, conversation_history_store],
        outputs=[game_state_store, conversation_history_store, chatbot_component, game_setup_feedback, audio_output_setup, game_setup_block, main_game_block]
    )

    player_action_outputs = [
        game_state_store, conversation_history_store, chatbot_component, audio_output_game,
        player_action_input,
        player_action_input,
        game_ended_flag
    ]

    def handle_game_flow_after_turn(is_game_over_flag):
        global model_text
        api_text_model_configured = bool(model_text)

        if is_game_over_flag:
            return gr.update(visible=not api_text_model_configured), gr.update(visible=api_text_model_configured), gr.update(visible=False)
        else:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

    game_turn_event_args = {
        "fn": rodada_do_jogo_gradio,
        "inputs": [player_action_input, game_state_store, conversation_history_store, chatbot_component, game_ended_flag],
        "outputs": player_action_outputs,
        "show_progress": "full"
    }

    submit_action_button.click(**game_turn_event_args).then(
        fn=handle_game_flow_after_turn,
        inputs=[game_ended_flag],
        outputs=[api_key_section, game_setup_block, main_game_block],
        show_progress=False
    )

    player_action_input.submit(**game_turn_event_args).then(
        fn=handle_game_flow_after_turn,
        inputs=[game_ended_flag],
        outputs=[api_key_section, game_setup_block, main_game_block],
        show_progress=False
    )

demo.queue()
demo.launch(debug=True, share=True, inbrowser=True)